In [ ]:
!pip install sklearn-crfsuite
import pickle
# filename = '/content/drive/MyDrive/CRF_model/model6.sav'
filename = '/content/drive/MyDrive/CRF_model/model6.sav'
crf_model=pickle.load(open(filename,'rb'))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 5.2 MB/s 


In [ ]:
# !pip install flair

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from flair.models import SequenceTagger
# from flair.data import Sentence
# tagger = SequenceTagger.load('pos')
# words = []
# POS = []
# sentence = Sentence("Arjuna and Draupadi lived in Hastinapura")
# tagger.predict(sentence)
# pos_tags = sentence.to_tagged_string()
# print(pos_tags)
# pos_tags_split = pos_tags.split()
# print(pos_tags_split)
# for i in range(0, len(pos_tags_split), 2):
#   words.append([pos_tags_split[i],pos_tags_split[i+1]])

# print(words)

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')  
from nltk.tokenize import word_tokenize

from nltk import pos_tag
from nltk import RegexpParser
# sentence = "Barack, Obama was born in Hawaii"

# text = word_tokenize(sentence)
# words = pos_tag(text)
# print("After Token:",words)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# s = words
# print(s)
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    # other than first word 
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]



In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# document = 'Barack Obama was born in Hawaii. Richard Manning wrote this sentence.'
# dict = {'subject': 'Barack Obama', 'relation': 'was', 'object': 'born'}
# document_name = 'Barack Obama was born in Hawaii. Richard Manning wrote this sentence.'
tokens = []
relations = []

In [ ]:
final_dict = []

In [ ]:
from nltk.tokenize import word_tokenize
import pandas as pd

df = pd.read_csv('/content/tag_crf.csv')

# df = df[:2]
for index, row in df.iterrows():
    tokens = []
    relations = []
    document = row['document']    
    text = row['subject']
    tok_temp = {}
    relation_temp={}

    #ner tag
    words = pos_tag(document)
    # print("After Token:",words)
    s = words
    ip = sent2features(s)
    # print([ip])
    pred = crf_model.predict([ip])
    # print(pred)

    # text = df['subject']
    start = document.find(text)
    end = start + len(text) 
    # print(start)
    # print(end)
    # print(text)
    doc_split = word_tokenize(document)
    text_split = word_tokenize(text)
    token_start = [i for i, e in enumerate(doc_split) if e == text_split[0]]
    if len(token_start) == 0:
      continue
    token_start = token_start[0]
    # token_start = doc_split.index(text_split[0])
    token_end = [i for i, e in enumerate(doc_split) if e == text_split[-1]]
    if len(token_end) == 0:
      continue
    token_end = token_end[0]
    if token_start == -1 or token_end == -1:
      continue
    if start == -1 or end == -1:
      continue
    print(token_start)
    print(token_end)
    tok_temp['text'] = text
    tok_temp['start'] = start
    tok_temp['end'] = end
    tok_temp['token_start'] = relation_temp['head'] = token_start
    tok_temp['token_end'] = token_end


    tok_temp['entityLabel'] = pred[0][token_start]
    tokens.append(tok_temp)

    tok_temp = {}
    text = row['object']
    start = document.find(text)
    end = start + len(text)
    print(start)
    print(end)
    print(text)
    print(document)
    doc_split = word_tokenize(document)
    text_split = word_tokenize(text)
    print(text_split[0])
    token_end = 0
    token_start = [i for i, e in enumerate(doc_split) if e == text_split[0]]
    if len(token_start) == 0:
      continue
    token_start = token_start[0]
    # token_start = doc_split.index(text_split[0])
    token_end = 0
    token_end = [i for i, e in enumerate(doc_split) if e == text_split[-1]]
    if len(token_end) == 0:
      continue
    token_end = token_end[0]
    if token_start == -1 or token_end == -1:
      continue
    if start == -1 or end == -1:
      continue
    print(token_start)
    print(token_end)
    tok_temp['text'] = text
    tok_temp['start'] = start
    tok_temp['end'] = end
    tok_temp['token_start'] = relation_temp['child']= token_start
    tok_temp['token_end'] = token_end
    tok_temp['entityLabel'] = pred[0][token_start]
    tokens.append(tok_temp)

    relation_temp['relationLabel'] = row['relation']

    relations.append(relation_temp)
    print(tokens)
    print(relations)
    temp = {}
    temp["document_name"] = document
    temp["document"] = document
    temp["tokens"] = tokens
    temp["relations"] = relations
    final_dict.append(temp)

In [ ]:
# print(tokens)
# print(relations)

In [ ]:
# temp = {}
# temp["document_name"] = document_name
# temp["document"] = document
# temp["tokens"] = tokens
# temp["relations"] = relations

In [ ]:
# print(temp)

In [ ]:
print(len(final_dict))

In [ ]:

    
with open("tagged_data_2.json", "w") as outfile:
    json.dump(final_dict, outfile)

In [ ]:
import json
json_object = json.dumps(final_dict, indent = 4) 
print(json_object)

In [ ]:
with open("train_data.txt", "w") as output:
    output.write(str(final_dict))